# Getting started with Ibis

In the [previous notebook](./00%20-%20Welcome.ipynb), we created a DuckDB database file with the [nycflights13 data](https://github.com/hadley/nycflights13). DuckDB is fast and runs locally, so it's handy for lots of use cases, including tutorials. Let's begin by importing Ibis and connecting to the database.

In [ ]:
import ibis

con = ibis.duckdb.connect("nycflights13.ddb", read_only=True)

**Note**: When you connect to a DuckDB database file, DuckDB creates a WAL file to prevent data corruption. If you see a `nycflights13.ddb.wal` file, you can safely ignore it. It will get cleaned up automatically.

Now we have a connection, we can start by looking around. Are there any tables in this database?

In [ ]:
con.list_tables()

Two, in fact! Let's take a look at the `flights` table first.

In [ ]:
flights = con.table("flights")

By default, you'll get a printable representation of the table schema, showing the name and data type of each column.

In [ ]:
flights

If we call the [`head`](https://ibis-project.org/reference/expression-tables.html#ibis.expr.types.relations.Table.head) method to peek at the data, you'll notice that we don't actually see data (yet); what's going on?

In [ ]:
flights.head()

Ibis has a deferred execution model. It builds up expressions based on what you ask it to do, and then executes those expressions on request.

In this case, our query isn't too involved; we want to see the first few rows of the `flights` table. We can do that by asking for the results of this query as a `pandas.DataFrame`:

In [ ]:
flights.head().to_pandas()

Or a `pyarrow.Table`:

In [ ]:
flights.head().to_pyarrow()

Or a `polars.DataFrame`:

In [ ]:
flights.head().to_polars()

We'll get into more detail about what Ibis is doing a bit later on. For now, the important point is that Ibis is deferred.

## Interactive mode

Remember when we said Ibis is deferred? Sometimes you want eager execution so you can explore a dataset. For the rest of this notebook, we'll turn on interactive mode, where Ibis will eagerly execute as much of the query as it needs to in order to show you the first 10 rows of the result.

In [ ]:
ibis.options.interactive = True

In interactive mode, we use `rich` to render the output inline:

In [ ]:
flights.head()

## Tables and columns

`flights` is a table!  A table is a collection of one or more columns, each with a specific datatype.

In [ ]:
flights

In [ ]:
type(flights)

We can look at a single column of that table using the column name as an attribute:

In [ ]:
flights.carrier

What kind of column is `carrier`? It's a `StringColumn`!

In [ ]:
type(flights.carrier)

## Ibis "verbs", or, stuff you can do to a table

The rest of this notebook covers some of the general methods you can use to alter the output of a particular table.

We'll cover, in order, `filter`, `select`, `drop`, `mutate`, `order_by`, `aggregate`, and `group_by`. Time to dive in!

## Filter

A filter allows you to view a subset of the rows in a table, based on some condition.

For instance, we might want to only view data for JetBlue flights:

In [ ]:
flights.filter(flights.carrier == "B6")

You can also combine multiple filters, across multiple columns.

We can subset the data down to JetBlue flights from JFK:

In [ ]:
expr = flights.filter((flights.carrier == "B6") & (flights.origin == "JFK"))
expr

Above, we combined two filters using `&`. You can also pass them in as individual arguments:

In [ ]:
expr = flights.filter(
    flights.carrier == "B6",
    flights.origin == "JFK",
)
expr

In [ ]:
ibis.to_sql(expr)

## Select

Filter filters, Select selects (there's a pattern here).
If you only want a subset of the columns in the original table, you can select
those columns explicitly.

You can refer to the columns using strings:

In [ ]:
flights.select("carrier", "origin", "dest")

Or you can use explicit references to the `Column` objects:

In [ ]:
flights.select(flights.carrier, flights.origin, flights.dest)

Or you can mix and match:

In [ ]:
flights.select("carrier", "origin", flights.dest)

## Drop

Drop is nearly the same as Select, but rather than explicitly choosing the columns to display, we explicitly choose the columns to _not_ display.

And as with `select`, you can specify the columns as strings:

In [ ]:
flights.drop("flight", "tailnum")

Or you can use explicit references to the `Column` objects:

In [ ]:
flights.drop(flights.flight, flights.tailnum)

Or you can mix and match:

In [ ]:
flights.drop("flight", flights.tailnum)

## Mutate

Everything we've seen so far has been subtractive—removing rows or columns. What about _adding_ columns?

That's what `mutate` is for! You can create a new column as a function of other existing columns (for example, converting units):

In [ ]:
flights.mutate(distance_km=flights.distance * 1.609)

Or you can create a new column and populate it with some literal value:

In [ ]:
flights.mutate(my_favorite_number=ibis.literal(41))

## On immutability

We've filtered, selected, dropped, and mutated this `flights` table quite a bit.

In [ ]:
flights

And yet, notice that none of our changes persist—the base table for our query isn't altered. The query (or expression) is a recipe of things to do with the base table (`flights`).

If you want to keep an expression around, you can assign it to a variable:

In [ ]:
expr = flights.select("carrier", "origin")
expr

**Note**: Every time you execute an expression (via interactive mode, or `to_pandas`, or similar), the entire expression gets executed, starting from the base table.  DuckDB is very fast and this dataset is very small, so the delay is unnoticeable, but for very large datasets, it might become more pronounced.  There is functionality to `cache` intermediate results that isn't covered in this tutorial, but you can [read more about it in the docs](https://ibis-project.org/reference/expression-tables.html#ibis.expr.types.relations.Table.cache).

## Method chaining

You can build up complicated queries by chaining together Ibis methods. The output of many Ibis methods is a table (just like `flights`!) and we can continue calling table methods until we're satisfied. Or until we end up with something that _isn't_ a table. More on that later.

In [ ]:
flights.select("carrier", "origin", "dest").drop("carrier")

Not the most complicated (or useful) query, but we'll see more soon.

### Exercise 1

Convert the `distance` column from miles to kilometers. For an approximate result, multiply by 1.609.

Two ways you might accomplish this:

- Chaining `.mutate` to create the new column and `.drop` to drop the original imperial column
- Using a single `.select` to create the new column as well as select the remaining columns

Try both ways below! How do they compare?

In [ ]:
# Convert the imperial units to metric, and drop the imperial columns.
# Try this using a `.mutate` and `.drop` call.
flights_metric_mutate_drop = flights

In [ ]:
# Convert the imperial units to metric, and drop the imperial columns.
# Try this using a single `.select` call.
flights_metric_select = flights

#### Solutions

In [ ]:
%load solutions/nb01_ex01_mutate_drop.py

In [ ]:
%load solutions/nb01_ex01_select.py

#### Does it matter which method you choose?

In this case, no. Sometimes, there might be a small difference in the generated SQL, but they will be semantically equivalent.

In [ ]:
ibis.to_sql(flights_metric_mutate_drop)

In [ ]:
ibis.to_sql(flights_metric_select)

In practice, small differences in the generated SQL don't make a difference. Any modern SQL execution engine will optimize variations to the same set of operations, and there will be no measurable performance difference.

## Order by

Want to order your data by a given column or columns?  Use `order_by`!

The default ordering direction is ascending:

In [ ]:
flights.order_by(flights.distance)

We can ask Ibis to sort in descending order, too.

In [ ]:
flights.order_by(flights.distance.desc())

Let's select out a subset of the columns to keep this a bit tidier.

In [ ]:
flights.order_by(flights.distance.desc()).select(
    "carrier", "origin", "dest", "distance"
)

You can also call `ibis.desc` on the column name to set the order direction:

In [ ]:
flights.order_by(ibis.desc("distance")).select("carrier", "origin", "dest", "distance")

## Aggregate

Ibis has several aggregate functions available to help summarize data.  All the old favorites are there: `mean`, `max`, `min`, `count`, `sum`...

You can aggregate a column by calling the method on that column:

In [ ]:
flights.distance.mean()

Or you can compute multiple aggregates using the `aggregate` method (also
available as `agg` for faster typing):

In [ ]:
flights.agg([flights.distance.mean(), flights.air_time.min()])

If you don't like the column names Ibis generates for you, choose your own!

In [ ]:
flights.agg(
    average_distance=flights.distance.mean(),
    shortest_air_time=flights.air_time.min(),
)

But aggregates really shine when paired with a `group_by`!

## Group by

`group_by` creates groupings of rows that have the same value for one or more columns.

But it doesn't do much on its own—you can pair it with `agg` to get a result.

In [ ]:
flights.group_by("carrier").agg()

Without any aggregate function specified, we get the distinct values of the grouped column.

We can add a second column to the `group_by` to get the distinct pairs across both columns:

In [ ]:
flights.group_by(["carrier", "origin"]).agg()

Now, if we add an aggregation function to that, we start to really open things up.

In [ ]:
flights.group_by(["carrier", "origin"]).agg(flights.distance.mean())

By adding that `mean` to the `aggregate`, we now have a concise way to calculate aggregates over each of the distinct groups in the `group_by`. And we can calculate as many aggregates as we need.

In [ ]:
flights.group_by(["carrier", "origin"]).agg(
    [flights.distance.mean(), flights.air_time.min()]
)

If we need more specific groups, we can add to the `group_by`.

In [ ]:
flights.group_by(["carrier", "origin", "dest"]).agg(
    [flights.distance.mean(), flights.air_time.min()]
)

## Cast

Sometimes when you parse data, _especially_ from CSVs, the types get a bit messed up. Or you might be loading in a `parquet` file where everything is defined as a `string`. We can clean that up pretty quickly.

You can cast from floats to ints:

In [ ]:
(flights.distance * 1.609).cast("int32")

And from ints to floats:

In [ ]:
flights.year.cast("float64")  # this is a terrible idea

You can cast numeric columns to strings:

In [ ]:
flights.year.cast("str")  # or "string"

And numeric strings to numbers:

In [ ]:
flights.year.cast("str").cast("int64")

But Ibis will yell if you try to cast a non-numeric string to a number:

In [ ]:
flights.carrier.cast("int32")

If we know that a column _should_ have a particular data type, but don't want a few bad apples (rows) to spoil the bunch, `try_cast` will fall back to `NULL` or `NaN` for values where the cast fails:

In [ ]:
flights.arr_delay.try_cast(int)

## Drop NA

Does what it says on the box—drop the `NULL`s from a dataset.

In [ ]:
flights.dropna()

## Exercises

Time to use what we've learned to answer some flight questions.

### Exercise 2

Which airlines (`carrier`) had the longest average arrival delays (`arr_delay`) in June 2013?

#### Solution

Note that there are several ways these queries could be written—it's fine if your solution doesn't look like ours, as long as the results are the same.

In [ ]:
%load solutions/nb01_ex02.py

### Exercise 3

Which NYC airport has the lowest percentage of outbound flights arriving 30 or more minutes late?

#### Solution

In [ ]:
%load solutions/nb01_ex03.py

## A brief digression on the SQL Ibis generates

Maybe you've heard that SQL has a standard?  This is true, and also misleading. The SQL standard is more of a suggestion, and there are myriad SQL _dialects_.

Ibis compiles expressions into the appropriate SQL dialect for the backend you are using. In this case, we started with a DuckDB table, so we get DuckDB SQL:

In [ ]:
ibis.to_sql(flights_metric_mutate_drop)

But if you want to use a _different_ dialect, you can pass the dialect name:

In [ ]:
ibis.to_sql(flights_metric_mutate_drop, dialect="postgres")

## Join

No dataframe library is complete without joins! Ibis supports several kinds of joins between table expressions: `inner_join`, `cross_join`, `left_join`, `outer_join`, `semi_join`, and `anti_join`. The `join` table method is, by default, the same as `inner_join`.

Remember the other table in our database?

In [ ]:
weather = con.table("weather")
weather

We can join the two tables on the `origin` column:

In [ ]:
flights.join(weather, "origin")

Of course, we should only join on the weather at the time corresponding to each flight:

In [ ]:
flights.join(weather, ["origin", "time_hour"])

The `on` condition can also be specified as an expression, which is particularly useful if you have columns with different names or non-equi-join logic.

In [ ]:
renamed = weather.rename(location="origin")
flights.join(
    renamed,
    (flights.origin != renamed.location) & (flights.time_hour == renamed.time_hour),
)

The Ibis `join` syntax is quite expressive, so we won't cover all the variations now; for more examples, read the [docs](https://ibis-project.org/reference/expression-tables#ibis.expr.types.relations.Table.join).